In [ ]:
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

# Use Your Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "<your-client-id>"
CLIENT_SECRET = "<your-client-secret>"

# Define Oath scopes with read only access
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"

# Redirect URI to open Authorization Code Window in Browser
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Build URL that generates Authorization Code
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()

#Print Authorization URL
print("Go to the following link in your browser: " + authorize_url)

auth_code = input("Enter your Authorization Code here:")

#get credentials to build GSC API service
credentials = flow.step2_exchange(auth_code)
print("The credentials are genereted successfully")

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()

#authorize credentials
creds = credentials.authorize(http)

#building a service to access various features of GSC API
webmasters_service = build('webmasters', 'v3', http=creds)

In [ ]:
# Get a list of sites in your Google Search Console Account
site_list = webmasters_service.sites().list().execute()

print(site_list)

In [ ]:
# Configure the dates and multiple urls of your website that you want to check.

site_url = 'https://www.example.com/'

# URLs provided as a multi-line string
url_list = """
https://www.example.com/home
https://www.example.com/pages/1
https://www.example.com/pages/2
"""

# Start and end dates for the data
start_date = '<YYYY-MM-DD>'
end_date = '<YYYY-MM-DD>'

In [ ]:
# Get Clicks & Impressions for multiple links

import csv

# Convert the multi-line string to a list of URLs
urls = [url.strip() for url in url_list.strip().split('\n') if url.strip()]

# Open a CSV file to write the results
with open('search_console_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['Page URL', 'Clicks', 'Impressions'])

    # Loop through the list of URLs and fetch data
    for url in urls:
        request = {
            'startDate': start_date,
            'endDate': end_date,
            'dimensions': ['page'],
            'dimensionFilterGroups': [{
                'filters': [{
                    'dimension': 'page',
                    'operator': 'equals',
                    'expression': url
                }]
            }],
            'rowLimit': 10000,  # Adjust based on how many results you expect
            'metrics': [{'expression': 'clicks'}, {'expression': 'impressions'}]
        }

        response = webmasters_service.searchanalytics().query(siteUrl=site_url, body=request).execute()

        # Write data to the CSV file
        if 'rows' in response:
            for row in response['rows']:
                writer.writerow([row['keys'][0], row['clicks'], row['impressions']])
        else:
            writer.writerow([url, 0, 0])  # Write zeros if no data is available